In [10]:
import sys
print(sys.executable)

/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/bin/python3


In [11]:
%%time
!python finetune-starcoder-1b.py\
--model_path="Salesforce/codegen-350M-multi"\
--seq_length=512\
--batch_size=2\
--eos_token_id=0\
--save_freq=2000\
--seed=42

python: can't open file '/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/codegen/finetune-starcoder-1b.py': [Errno 2] No such file or directory
CPU times: user 2.73 ms, sys: 42 ms, total: 44.7 ms
Wall time: 197 ms


# Load the model

In [1]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')


Device: cuda:0


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi", load_in_8bit=True, torch_dtype=torch.bfloat16, device_map='auto')
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi", device_map='auto')

Overriding torch_dtype=torch.bfloat16 with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


# Prepare data



In [3]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={
    'train': '/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/data/nspm-fine-tuning/train.csv', 
    'val': '/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/data/nspm-fine-tuning/val.csv'
})


print(dataset)

Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-3615beb83b6f428b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'prompt', 'completion'],
        num_rows: 100000
    })
    val: Dataset({
        features: ['Unnamed: 0', 'prompt', 'completion'],
        num_rows: 20000
    })
})


In [4]:
print(dataset['train'][0]['prompt'])
print(dataset['train'][0]['completion'])

# write a SPARQL query for:
# How many ICD9 did friedreich's ataxia have ?
select count(*) as ?x where{dbr:Friedreich's_ataxia dbo:icd9 ?x }


In [5]:
from datasets import load_dataset
from torch.utils.data import Dataset
import copy

class PrepareDataset(Dataset):
    def __init__(self, data_path=str, split='train', size=None, max_seq_length=2048):
        self.dataset = load_dataset('csv', data_files={
            'train': data_path + 'train.csv',
            'val': data_path + 'val.csv',
            'test': data_path + 'test.csv',
        })[split]  
        
        if size:
            self.dataset = self.dataset.select(range(size))
        
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        item = self.dataset[index]
        prompt = torch.tensor(tokenizer.encode(item['prompt']), dtype=torch.int64)
        example = item['prompt'] + item["completion"]
        example = tokenizer.encode(example)
        example.append(tokenizer.eos_token_id)

        if len(example) > self.max_seq_length:
            example = example[:self.max_seq_length]

        example = torch.tensor(example, dtype=torch.int64)
        example = torch.nn.functional.pad(example, (0, self.max_seq_length - len(example)), 'constant', 0)
        
        labels = example.clone()
        labels[:len(prompt)] = -100

        return {"prompt": example, "labels": labels}

In [6]:

data_path = "/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/data/nspm-fine-tuning/"

train_dataset = PrepareDataset(data_path=data_path, split='train', size=None, max_seq_length=512)
val_dataset = PrepareDataset(data_path=data_path, split='val', max_seq_length=512)
test_dataset = PrepareDataset(data_path=data_path, split='test', max_seq_length=512)

Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-72c2059406228e0d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-72c2059406228e0d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-72c2059406228e0d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
train_dataset

# Prepare PEFT for model training

In [ ]:
from peft import pr

In [9]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_int8_training,
)


model.train()

model = prepare_model_for_int8_training(model)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules = ["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/lib/python3.11/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


ValueError: Target modules ['q_proj', 'v_proj'] not found in the base model. Please check the target modules and try again.

# Finetune the model with Huggingface trainer

In [ ]:
from transformers import default_data_collator, Trainer, TrainingArguments

output_dir = "training_run"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'gradient_checkpointing': False,
}


training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=5,
    optim="adamw_torch_fused",
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=default_data_collator,
)

# Start training
trainer.train()

# Save model checkpoint

In [ ]:
model.save_pretrained(output_dir)

# Evaluate the finetuned model

In [ ]:
model.eval()

In [ ]:
text = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many stadiums in total?

SELECT"""

model_input = tokenizer(text, return_tensors="pt").to("cuda")

generated_ids = model.generate(**model_input, max_new_tokens=100)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))